#Importing

In [ ]:
import cv2
import numpy as np

In [ ]:
from google.colab.patches import cv2_imshow

#center initializing

because mean-shift is computationaly expensive, instead of assuming each pixel as a center in the first step, we assume some centers uniformly. Also, we calculate our 2D window (its bandwidth) as below :

an 2S*2S window with:

$
S = \sqrt{ \frac{h*w}{K}}
$

in which K is the number of centers and h and w are the dimensions of the image.

In [ ]:
def initialize(K, img):
  h, w = img[:, : ,0].shape
  cluster = []
  S = int(np.sqrt(h*w / K))
  i = S / 2
  j = S / 2
  cnt = 0
  while i < h:
    while j < w:
      #cluster[cnt, 0] = i
      #cluster[cnt, 1] = j
      cluster.append((int(i), int(j)))
      j += S
      cnt += 1
    j = S / 2
    i += S
  return np.array(cluster)

#center updating

for the first iteration we just calculate the mean of the feature vector (rgb values) in each window.
But for other iterations we check if each pixel's distance with the cluster's center is less than an specific value (here by examination we put it 60), if this condition has been met, we add this pixel to the cluster and use it for averaging the rgb values of the cluster so we can compute the new center.

In [ ]:
def center_update(img, center, K, flag=1):
  h, w = img[:, :, 0].shape
  S = int(np.sqrt(h*w / K))
  new_center = []
  res = np.zeros((h, w))
  c_count = 0
  if flag == 0:
    for c in center:
      #print(c_count)
      low_I  = c[0] - S
      high_I = c[0] + S
      low_J  = c[1] - S
      high_J = c[1] + S
      if low_I<0:
        low_I = 0
      if high_I > h:
        high_I = h
      if low_J <0:
        low_J = 0
      if high_J > w:
        high_J = w

      mean_r = int(round(np.mean(img[low_I:high_I, low_J:high_J, 0])))
      mean_g = int(round(np.mean(img[low_I:high_I, low_J:high_J, 1])))
      mean_b = int(round(np.mean(img[low_I:high_I, low_J:high_J, 2])))
      new_center.append((mean_r, mean_g, mean_b))
  else:
    for c in center:
      sum_r = 0
      sum_g = 0
      sum_b = 0
      num = 0
      for i in range(h):
        for j in range(w):
          if(np.linalg.norm(img[i, j, :] - c) < 60):
            sum_r += img[i, j, 0]
            sum_g += img[i, j, 1]
            sum_b += img[i, j, 2]
            num   += 1
            res[i, j] = c_count
      r = int(round(sum_r / num))
      g = int(round(sum_g / num))
      b = int(round(sum_b / num))
      new_center.append((r, g, b))
      c_count += 1




  return np.array(new_center), res

#Running the algorithm

by examination we set the number of initial centers 80. because the algorithm is computationaly expensive we just run 3 iterations. (although we can define an error function and compute the difference between current centers and previous ones and set an threshold, but here we just put the max_iter = 3)

In [73]:
img = cv2.imread('/content/Furano.jpg')
center0 = initialize(80, img)
new_c, res = center_update(img, center0, 80, 0)

In [74]:
img1 = img.copy()
for c in center0:
  img1 = cv2.circle(img1, (c[1], c[0]), radius=2, color=(0, 0, 255), thickness=2)

cv2_imshow(img1)

Output hidden; open in https://colab.research.google.com to view.

In [75]:
iter = 3

for i in range(iter):
  print(i)
  new_c, res = center_update(img, new_c, 80, 1)

0
1
2


At last for each cluster, we find its pixels (using res matrix) and put the mean value (the vlues of the center) in each pixel.

In [76]:
img1 = img.copy()
h, w = img[:, :, 0].shape
for l in range(40):
  for i in range(h):
    for j in range(w):
      if(res[i, j] == l):
        img1[i, j, 0] = new_c[l, 0]
        img1[i, j, 1] = new_c[l, 1]
        img1[i, j, 2] = new_c[l, 2]

In [77]:
cv2_imshow(img1)

Output hidden; open in https://colab.research.google.com to view.